<a href="https://colab.research.google.com/github/mukkatharun/advance-deep-learning-assignments/blob/main/Assignment5_Continual%20ML%20and%20Active%20Learning/continual_learning_using_avalanche_library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install avalanche-lib==0.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 582 kB 28.2 MB/s 
     |████████████████████████████████| 529 kB 58.4 MB/s 
     |████████████████████████████████| 532 kB 57.7 MB/s 
     |████████████████████████████████| 427 kB 53.2 MB/s 
     |████████████████████████████████| 1.9 MB 57.3 MB/s 
     |████████████████████████████████| 166 kB 62.6 MB/s 
     |████████████████████████████████| 182 kB 60.6 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 166 kB 62.7 MB/s 
     |████████████████████████████████| 162 kB 66.3 MB/s 
     |████████████████████████████████| 162 kB 70.5 MB/s 
     |████████████████████████████████| 158 kB 76.2 MB/s 
     |████████████████████████████████| 157 kB 63.9 MB/s 
     |████████████████████████████████| 157 kB 73.3 MB/s 
     |████████████████████████████████| 157 kB 70.2 MB/s 
     |███████████████████████████

I am using the Avalanche library to create end to end demo and i am choosing SimpleMLP modeling on PermutedMNIST dataset


Importing necessary libraries

In [3]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import Naive

In [6]:
 # Configure and check for cuda if available use cude gpu else cpu

 device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 device


device(type='cuda', index=0)

In [7]:
model = SimpleMLP(num_classes = 10) #instantiating multi-layer perceptron model.

Prepare the benchmark dataset. Split the dataset into train and test sets.



In [9]:
# Benchmark instantiation
perm_mnist = PermutedMNIST(n_experiences=3)
# split stream into train and test set
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



Instantiate optimizer and loss functions.


In [10]:
# define optimizer and loss function for training
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

In [11]:
# Instantiate Continual learning strategy
strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)


Perform training, evaluate the model and log results.


In [12]:
results = []
# train and test epochs
for train_task in train_stream:
    # training
    strategy.train(train_task, num_workers=4)
    # evaluate the model and store the results
    results.append(strategy.eval(test_stream))

-- >> Start of training phase << --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 1875/1875 [00:41<00:00, 45.44it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4890
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8604
100%|██████████| 1875/1875 [00:40<00:00, 46.16it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2573
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9253
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:07<00:00, 43.58it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1820
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9491
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 313/313 [00:06<00:00, 45.56it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task001/Exp001 = 2.5482
	Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001 = 0.1142
-- S

In [13]:
results[0]

{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9253,
 'Loss_Epoch/train_phase/train_stream/Task000': 0.2572722277760506,
 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9491,
 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.1820282697573304,
 'Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001': 0.1142,
 'Loss_Exp/eval_phase/test_stream/Task001/Exp001': 2.548235055923462,
 'Top1_Acc_Exp/eval_phase/test_stream/Task002/Exp002': 0.1825,
 'Loss_Exp/eval_phase/test_stream/Task002/Exp002': 2.2845621604919435,
 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.9491,
 'Top1_Acc_Stream/eval_phase/test_stream/Task001': 0.1142,
 'Top1_Acc_Stream/eval_phase/test_stream/Task002': 0.1825,
 'Loss_Stream/eval_phase/test_stream/Task000': 0.1820282697573304,
 'Loss_Stream/eval_phase/test_stream/Task001': 2.548235055923462,
 'Loss_Stream/eval_phase/test_stream/Task002': 2.2845621604919435}

In [15]:
results

[{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9253,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.2572722277760506,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9491,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.1820282697573304,
  'Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001': 0.1142,
  'Loss_Exp/eval_phase/test_stream/Task001/Exp001': 2.548235055923462,
  'Top1_Acc_Exp/eval_phase/test_stream/Task002/Exp002': 0.1825,
  'Loss_Exp/eval_phase/test_stream/Task002/Exp002': 2.2845621604919435,
  'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.9491,
  'Top1_Acc_Stream/eval_phase/test_stream/Task001': 0.1142,
  'Top1_Acc_Stream/eval_phase/test_stream/Task002': 0.1825,
  'Loss_Stream/eval_phase/test_stream/Task000': 0.1820282697573304,
  'Loss_Stream/eval_phase/test_stream/Task001': 2.548235055923462,
  'Loss_Stream/eval_phase/test_stream/Task002': 2.2845621604919435},
 {'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9253,
  'Loss_Epoch/trai